In [ ]:
DOCKER_REGISTRY = 'index.docker.io/dejangolubovic'
my_namespace = 'dejan-golubovic'

In [ ]:
num_chief = 1 #number of Chief in TFJob 
num_ps = 1  #number of PS in TFJob 
num_workers = 2  #number of Worker in TFJob 
model_dir = "/mnt"
export_path = "/mnt/export" 
train_steps = "1000"
batch_size = "100"
learning_rate = "0.01"
pv_name = 'pvc-f8cc766b-aef7-4456-99bb-8c99621094af'
pvc_name = 'dejan-fairing-pvc-2'

In [ ]:
import uuid
from kubeflow import fairing   
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

tfjob_name = f'mnist-training-{uuid.uuid4().hex[:4]}'
print(tfjob_name)
output_map =  {
    "Dockerfile": "Dockerfile",
    "mnist.py": "mnist.py"
}

context_source = ''

command=["python",
         "/opt/mnist.py",
         "--tf-model-dir=" + model_dir,
         "--tf-export-dir=" + export_path,
         "--tf-train-steps=" + train_steps,
         "--tf-batch-size=" + batch_size,
         "--tf-learning-rate=" + learning_rate]

minio_context_source = MinioContextSource(
            endpoint_url='http://minio-service.kubeflow:9000/',
            minio_secret='minio',
            minio_secret_key='minio123',#
            region_name='region')

fairing.config.set_preprocessor('python', command=command, path_prefix="/app", output_map=output_map)
fairing.config.set_builder(name='cluster', registry=DOCKER_REGISTRY, \
                           context_source=minio_context_source, cleanup=True, \
                          pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.set_deployer(name='tfjob', namespace=my_namespace, stream_log=False, job_name=tfjob_name,
                            chief_count=num_chief, worker_count=num_workers, ps_count=num_ps, 
                            pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.run()
#fairing.config.run()

In [ ]:
#tfjob_name = 'mnist-training-565f'
tfjob_namespace = 'dejan-golubovic'

In [ ]:
from kubeflow.tfjob import TFJobClient
tfjob_client = TFJobClient()

tfjob_client.get(tfjob_name, namespace=tfjob_namespace)

In [ ]:
tfjob_client.wait_for_job(tfjob_name, namespace=tfjob_namespace, watch=True)

In [ ]:
tfjob_client.is_job_succeeded(tfjob_name, namespace=tfjob_namespace)

In [ ]:
#tfjob_client.get_logs(tfjob_name, namespace=tfjob_namespace)

In [ ]:
from kubeflow.fairing.deployers.kfserving.kfserving import KFServing
isvc_name = f'dejan-mnist-service-{uuid.uuid4().hex[:4]}'
print(isvc_name)
isvc = KFServing('tensorflow', namespace=tfjob_namespace, isvc_name=isvc_name,
                 default_storage_uri='pvc://' + pvc_name + '/export')
isvc.deploy(isvc.generate_isvc())

In [ ]:
from kfserving import KFServingClient
kfserving_client = KFServingClient()
kfserving_client.get(namespace=tfjob_namespace)

In [ ]:
mnist_isvc = kfserving_client.get(isvc_name, namespace=tfjob_namespace)
mnist_isvc_name = mnist_isvc['metadata']['name']
mnist_isvc_endpoint = mnist_isvc['status'].get('url', '')
print("MNIST Service Endpoint: " + mnist_isvc_endpoint)

In [ ]:
#ISTIO_CLUSTER_IP=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.clusterIP}'
#CLUSTER_IP=ISTIO_CLUSTER_IP[0]
CLUSTER_IP = '10.254.73.13'
MODEL_HOST=f"Host: {mnist_isvc_name}.{tfjob_namespace}.example.com"
print(MODEL_HOST)
print('curl -v -H'+MODEL_HOST+' http://'+CLUSTER_IP+'/v1/models/'+mnist_isvc_name+':predict -d @./input.json')
!curl -v -H "{MODEL_HOST}" http://{CLUSTER_IP}/v1/models/{mnist_isvc_name}:predict -d @./input.json

In [ ]:
tfjob_client.delete(tfjob_name, namespace=my_namespace)

In [ ]:
kfserving_client.delete(isvc_name, namespace=my_namespace)